In [22]:
import openai
import os
import dotenv
dotenv.load_dotenv()

True

In [23]:
client = openai.Client(api_key=os.getenv("openaiKey"))

In [24]:
vector_stores = client.beta.vector_stores.create(
    name="tutor de apostilas"
)

In [25]:
file = ["LLM.pdf"]
file_stream = [open(f,"rb") for f in file]
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_stores.id,files=file_stream
)

In [26]:
file_batch.status

'completed'

In [27]:
file_batch.file_counts

FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)

In [28]:
assistant = client.beta.assistants.create(
    name="tutor apostila",
    instructions="você é um tutor especializados em tecnologias emergentes. você sabe responder perguntas sobre LLMs com OpenAI,huggingface,etc. \
        caso você  nao encontre as respostas, seja sincero e fale que não sabe responder.",

    tools=[{"type":"file_search"}],
    tool_resources={"file_search":{"vector_store_ids":[vector_stores.id]}},
    model="gpt-4-turbo-preview"

    )

In [29]:
pergunta = "conforme o documento, o que e o huggingface ?"

In [30]:
thread = client.beta.threads.create()
mensagens = client.beta.threads.messages.create(thread_id=thread.id,role="user",content=pergunta)

In [31]:
run = client.beta.threads.runs.create(thread_id=thread.id,assistant_id=assistant.id,instructions="nome de usuario premium")


In [32]:
import time
while run.status in ["queued","in_progress","cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)

In [33]:
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(thread_id=thread.id)
    print(mensagens)
else:
    print(f"erro{run.status}")

SyncCursorPage[Message](data=[Message(id='msg_Y8cKwdAg20jVUbW5LHYxl31h', assistant_id='asst_MjvQmAJZxWsmVAyFboRITXoG', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=510, file_citation=FileCitation(file_id='file-9FkF6uoC6VjRXtktU198M8'), start_index=497, text='【4:0†LLM.pdf】', type='file_citation')], value='Hugging Face é descrita no documento como uma comunidade de código aberto que reúne centenas de milhares de modelos contribuídos que podem ajudar a resolver muitos casos de uso específicos, como geração de texto, resumo e classificação. Ela é destacada por seu crescimento explosivo nos últimos anos dentro do campo aberto de modelos de linguagem, mostrando uma rápida aproximação do desempenho dos modelos proprietários, apesar de ainda não alcançar o desempenho de modelos avançados como o GPT-4【4:0†LLM.pdf】.'), type='text')], created_at=1740592417, incomplete_at=None, incomplete_details=None, metadata={}, object='thr

In [34]:
mensagens.data[0].content[0].text.value

'Hugging Face é descrita no documento como uma comunidade de código aberto que reúne centenas de milhares de modelos contribuídos que podem ajudar a resolver muitos casos de uso específicos, como geração de texto, resumo e classificação. Ela é destacada por seu crescimento explosivo nos últimos anos dentro do campo aberto de modelos de linguagem, mostrando uma rápida aproximação do desempenho dos modelos proprietários, apesar de ainda não alcançar o desempenho de modelos avançados como o GPT-4【4:0†LLM.pdf】.'